# U.S. Medical Insurance Costs

In [43]:
import csv

insurance_array = list()

# Open and save data from insurance.csv to Python variable
with open("insurance.csv", newline='') as insurance_csv:
  insurance_data = csv.DictReader(insurance_csv)
  for row in insurance_data:
    insurance_array.append(row)

print(len(insurance_array), "\n", insurance_array[0], "\n", insurance_array[1200])
    


1338 
 {'age': '19', 'sex': 'female', 'bmi': '27.9', 'children': '0', 'smoker': 'yes', 'region': 'southwest', 'charges': '16884.924'} 
 {'age': '37', 'sex': 'male', 'bmi': '24.32', 'children': '2', 'smoker': 'no', 'region': 'northwest', 'charges': '6198.7518'}


## Sort by sex

In [65]:
# Sort by sex
sex_sorted = dict()
male_data = list()
female_data = list()
for p in insurance_array:
    if p['sex'] == 'female':
        female_data.append(p)
    if p['sex'] == 'male':
        male_data.append(p)

sex_sorted.update({'male': male_data, 'female': female_data})
print("# of Female datapoints: " + str(len(sex_sorted['male'])))
print("# of Female datapoints: " + str(len(sex_sorted['female'])))

# of Female datapoints: 676
# of Female datapoints: 662


### Average BMI of Female and Male persons in dataset

In [35]:
#Find average BMI for males and females
def average_bmi(dataset):
    total_bmi = 0
    total_datapoints = len(dataset)

    for p in dataset:
        total_bmi += float(p['bmi'])
    
    average_bmi = total_bmi / total_datapoints
    
    return round(average_bmi,4)

avg_female_bmi = average_bmi(female_data)
avg_male_bmi = average_bmi(male_data)

print(f"Average Female BMI: {avg_female_bmi}\nAverage Male BMI: {avg_male_bmi}")


Average Female BMI: 30.3777
Average Male BMI: 30.9431


### Average Cost: Male vs. Female

## Sort By Region (Dictionary)

In [49]:
def sort_by_region(dataset):
    region_dict = dict()

    for s in dataset:
      if s['region'] not in region_dict:
         region_dict[s['region']] = [s]
      else:
         region_dict[s['region']].append(s)
    
    return region_dict


region_sorted = sort_by_region(insurance_array)
print(region_sorted.keys())



dict_keys(['southwest', 'southeast', 'northwest', 'northeast'])
[{'age': '18', 'sex': 'male', 'bmi': '33.77', 'children': '1', 'smoker': 'no', 'region': 'southeast', 'charges': '1725.5523'}, {'age': '28', 'sex': 'male', 'bmi': '33', 'children': '3', 'smoker': 'no', 'region': 'southeast', 'charges': '4449.462'}, {'age': '31', 'sex': 'female', 'bmi': '25.74', 'children': '0', 'smoker': 'no', 'region': 'southeast', 'charges': '3756.6216'}, {'age': '46', 'sex': 'female', 'bmi': '33.44', 'children': '1', 'smoker': 'no', 'region': 'southeast', 'charges': '8240.5896'}, {'age': '62', 'sex': 'female', 'bmi': '26.29', 'children': '0', 'smoker': 'yes', 'region': 'southeast', 'charges': '27808.7251'}, {'age': '56', 'sex': 'female', 'bmi': '39.82', 'children': '0', 'smoker': 'no', 'region': 'southeast', 'charges': '11090.7178'}, {'age': '27', 'sex': 'male', 'bmi': '42.13', 'children': '0', 'smoker': 'yes', 'region': 'southeast', 'charges': '39611.7577'}, {'age': '18', 'sex': 'male', 'bmi': '34.1', 

### Average cost by region:

In [61]:
def average_cost_by_region(region_sorted_data):
  average_by_region = dict()

  for region in region_sorted_data:
      total_charges = 0
      total_datapoints = len(region_sorted_data[region])

      for p in region_sorted[region]:
          total_charges += float(p['charges'])
      
      average_by_region[region] = round((total_charges / total_datapoints), 2)

  return average_by_region

average_by_region = average_cost_by_region(region_sorted)
print(average_by_region)
    

{'southwest': 12346.94, 'southeast': 14735.41, 'northwest': 12417.58, 'northeast': 13406.38}
